<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#train-each-task-and-get-prediction" data-toc-modified-id="train-each-task-and-get-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>train each task and get prediction</a></span><ul class="toc-item"><li><span><a href="#country" data-toc-modified-id="country-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>country</a></span></li><li><span><a href="#politics" data-toc-modified-id="politics-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>politics</a></span></li><li><span><a href="#tod" data-toc-modified-id="tod-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>tod</a></span></li><li><span><a href="#age" data-toc-modified-id="age-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>age</a></span></li><li><span><a href="#education" data-toc-modified-id="education-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>education</a></span></li><li><span><a href="#ethnic" data-toc-modified-id="ethnic-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>ethnic</a></span></li><li><span><a href="#gender" data-toc-modified-id="gender-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>gender</a></span></li></ul></li><li><span><a href="#combine-all-prediction" data-toc-modified-id="combine-all-prediction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>combine all prediction</a></span></li></ul></div>

In [ ]:
%run ./PASTEL_MTL_training_utils.ipynb

In [ ]:
import wandb
os.environ['WANDB_NOTEBOOK_NAME'] = 'PASTEL single task on unmasked set.ipynb'
os.environ['WANDB_PROJECT']="PASTEL single task on unmasked set"
os.environ['WANDB_ENTITY']="fsu-dsc-cil"
os.environ['WANDB_DIR']="/scratch/data_jz17d/wandb_tmp/"

In [ ]:
def predict_logits_labels(trainer, dataset, task_name):
    pred = trainer.predict(dataset)
    df = dataset.df.copy()
    logits = pred.predictions[task_name]
    labels = logits.argmax(-1)
    df['logits'] = logits.tolist()
    df[f'predicted_{task_name}'] = labels.tolist()
    return df

# train each task and get prediction

In [ ]:
# global settings
p=0.8
num_epoch=15
save_prediction = True

## country

In [ ]:
################# settings #################
selected_tasks = ['country']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
***** Running training *****
  Num examples = 6385
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, dist

Epoch,Training Loss,Validation Loss,Accuracy Country,F1 Country,Precision Country,Recall Country
1,1.313700,0.114288,0.977892,0.329608,0.325964,0.333333
2,1.313700,0.107581,0.977892,0.329608,0.325964,0.333333
3,0.172300,0.172681,0.963613,0.344475,0.346353,0.343310
4,0.172300,0.204686,0.971087,0.340028,0.347853,0.338732
5,0.025200,0.218312,0.973644,0.342525,0.360677,0.340198
6,0.025200,0.238385,0.974745,0.340333,0.362027,0.338792
7,0.025200,0.245526,0.975257,0.341695,0.371039,0.339560
8,0.000600,0.249580,0.975886,0.336865,0.361935,0.336806
9,0.000600,0.254991,0.975886,0.335800,0.357771,0.336212
10,0.000200,0.257327,0.976004,0.336922,0.363673,0.336846


***** Running Evaluation *****
  Num examples = 25421
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-200
Configuration saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-200/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-200/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-200/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task country/20220923

Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-1600/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25421
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-1800
Configuration saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-1800/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task co

tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-3000/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-3000/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-2800] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /scratch/data_jz17d/result/PASTEL single task country/20220923-17:46:55/checkpoint-400 (score: 0.10758054256439209).
***** Running Prediction *****
  Num examples = 6385
  Batch size = 32


/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25421
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## politics

In [ ]:
################# settings #################
selected_tasks = ['politics']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Politics,F1 Politics,Precision Politics,Recall Politics
1,1.251900,0.974789,0.486500,0.350041,0.324084,0.384977
2,1.251900,0.961454,0.500117,0.360191,0.442626,0.399834
3,0.960600,1.064373,0.482431,0.410353,0.430978,0.412795
4,0.960600,1.457381,0.445257,0.415506,0.426777,0.431926
5,0.960600,1.899788,0.471866,0.425851,0.425904,0.426023
6,0.416200,2.516419,0.485522,0.411069,0.426216,0.412798
7,0.416200,2.847833,0.468344,0.415947,0.421203,0.417214
8,0.106900,3.253424,0.476053,0.421528,0.425584,0.421071
9,0.106900,3.465040,0.474409,0.424245,0.426462,0.424053
10,0.106900,3.368948,0.484505,0.419879,0.429967,0.419234


***** Running Evaluation *****
  Num examples = 25556
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-196
Configuration saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-196/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-196/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-196/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task politics/202

tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-1960/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-1960/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-1764] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25556
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-2156
Configuration saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-2156/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-2156/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task politics/20220923-18:14:58/checkpoint-2156/tokenize

***** Running Prediction *****
  Num examples = 25556
  Batch size = 32


## tod

In [ ]:
################# settings #################
selected_tasks = ['tod']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Tod,F1 Tod,Precision Tod,Recall Tod
1,1.749100,1.353066,0.447926,0.209758,0.249461,0.255770
2,1.749100,1.344156,0.456104,0.212646,0.260964,0.260374
3,1.343400,1.401648,0.405032,0.235131,0.270942,0.255489
4,1.343400,1.803065,0.353017,0.248724,0.281381,0.264400
5,0.725400,2.272214,0.347671,0.256616,0.264026,0.265415
6,0.725400,2.782790,0.371732,0.253984,0.277604,0.263957
7,0.725400,3.443300,0.371142,0.270854,0.277932,0.273546
8,0.143500,3.638229,0.379005,0.258481,0.267590,0.262939
9,0.143500,4.024141,0.388284,0.258392,0.270638,0.262818
10,0.070100,4.111552,0.385217,0.266739,0.275496,0.268168


***** Running Evaluation *****
  Num examples = 25435
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-200
Configuration saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-200/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-200/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-200/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint

Model weights saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2000/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2000/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-1800] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25435
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2200
Configuration saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2200/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task tod/20220923-18:42:26/checkpoint-2200/pytorch_model.bin
tokenizer config file saved in /s

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25435
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## age

In [ ]:
################# settings #################
selected_tasks = ['age']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Age,F1 Age,Precision Age,Recall Age
1,2.024000,1.459636,0.398530,0.073932,0.131372,0.125896
2,2.024000,1.391327,0.415628,0.164631,0.243831,0.172038
3,1.444500,1.490575,0.411344,0.164774,0.263879,0.176918
4,1.444500,1.755475,0.392948,0.198472,0.200025,0.199297
5,1.444500,2.352129,0.390000,0.200063,0.231782,0.198415
6,0.707000,2.859619,0.389961,0.198583,0.224027,0.196832
7,0.707000,3.315315,0.379073,0.205988,0.237101,0.205179
8,0.135900,3.746656,0.377973,0.204286,0.237902,0.204678
9,0.135900,3.863787,0.389175,0.204100,0.222519,0.201419
10,0.135900,4.014277,0.378719,0.217466,0.229516,0.215580


***** Running Evaluation *****
  Num examples = 25441
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-199
Configuration saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-199/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-199/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-199/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint

Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1393] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25441
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1791
Configuration saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1791/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1791/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1791/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1791/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task age/20220923-19:09:46/checkpoint-1592] due to args.save_total_limit
***** Running Eva

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25441
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## education

In [ ]:
################# settings #################
selected_tasks = ['education']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Education,F1 Education,Precision Education,Recall Education
1,2.444800,1.615172,0.394124,0.139571,0.216514,0.165244
2,2.444800,1.611636,0.361878,0.178142,0.254431,0.183198
3,1.674100,1.607066,0.397772,0.185275,0.230656,0.184299
4,1.674100,1.892083,0.380668,0.189269,0.205374,0.191107
5,1.674100,2.514521,0.364114,0.198194,0.238764,0.194247
6,0.849800,3.020540,0.350698,0.183330,0.200035,0.188808
7,0.849800,3.370916,0.359838,0.204367,0.212950,0.200008
8,0.125900,3.850363,0.356386,0.190323,0.203167,0.191779
9,0.125900,4.067533,0.360270,0.198357,0.208224,0.197419
10,0.125900,4.288568,0.354700,0.200870,0.207072,0.202110


***** Running Evaluation *****
  Num examples = 25492
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-198
Configuration saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-198/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-198/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-198/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task educatio

  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1584
Configuration saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1584/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1584/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1584/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1584/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/checkpoint-1386] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25492
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task education/20220923-19:37:15/che

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25492
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## ethnic

In [ ]:
################# settings #################
selected_tasks = ['ethnic']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Ethnic,F1 Ethnic,Precision Ethnic,Recall Ethnic
1,2.073800,0.732880,0.823998,0.181246,0.174819,0.189549
2,2.073800,0.737943,0.827023,0.195441,0.291460,0.195738
3,0.824000,0.753929,0.825373,0.214168,0.321434,0.210010
4,0.824000,0.884629,0.817243,0.212105,0.294603,0.207276
5,0.824000,1.230417,0.798115,0.228884,0.276504,0.219622
6,0.330000,1.463447,0.802435,0.221225,0.275348,0.214236
7,0.330000,1.541822,0.795208,0.236036,0.272753,0.227990
8,0.050800,1.631372,0.801571,0.230119,0.282121,0.217932
9,0.050800,1.669639,0.802985,0.232775,0.294984,0.218726
10,0.050800,1.720805,0.816889,0.227942,0.315748,0.216032


***** Running Evaluation *****
  Num examples = 25460
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-199
Configuration saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-199/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-199/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-199/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:0

Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1592/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1393] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25460
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1791
Configuration saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1791/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1791/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1791/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task ethnic/20220923-20:04:34/checkpoint-1791/special_tokens_map.json


/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25460
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## gender

In [ ]:
################# settings #################
selected_tasks = ['gender']
############################################
# build args
my_training_args = MyTrainingArgs(selected_tasks=selected_tasks,
                                  model_name=f'PASTEL single task {selected_tasks[0]}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=num_epoch,
#                                   data_limit=30000,
                                 )
hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # either 'epoch' or 'steps'
    save_strategy = 'epoch',
    save_total_limit = 1,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,  # by default decide on loss, change it by setting metric_for_best_model
    report_to="wandb",
    run_name=f"single task bert on unmasked PASTEL-{selected_tasks[0]}",
)

# initialize model and dataset
model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_unmasked_train')
val_dataset = MyDataset(my_training_args, f'p={p}_masked_train')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train
trainer.train()

# get prediction and save to file
if save_prediction:
    df_train = predict_logits_labels(trainer, train_dataset, selected_tasks[0])
    df_val = predict_logits_labels(trainer, val_dataset, selected_tasks[0])
    df_combined = pd.concat([df_train, df_val], axis = 0)
    df_combined = df_combined.sort_values('Unnamed: 0')

    output_path = f'{data_folder}/pastel/processed/p={p}_predicted'
    Path(output_path).mkdir(parents=True, exist_ok=True)
    df_combined.to_csv(f'{output_path}/{selected_tasks[0]}.csv', index=False)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/re

Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropo

Epoch,Training Loss,Validation Loss,Accuracy Gender,F1 Gender,Precision Gender,Recall Gender
1,1.538900,0.614407,0.720137,0.415768,0.522393,0.421009
2,1.538900,0.603215,0.727240,0.448253,0.483211,0.444678
3,0.638400,0.674099,0.738013,0.444116,0.516560,0.441921
4,0.638400,0.855589,0.707391,0.445786,0.456279,0.443348
5,0.294000,1.327684,0.666785,0.432554,0.430010,0.435983
6,0.294000,1.638546,0.720571,0.447603,0.503298,0.441895
7,0.294000,1.863245,0.695908,0.439915,0.447327,0.437821
8,0.080800,2.037188,0.697644,0.444781,0.454738,0.441608
9,0.080800,2.170302,0.717533,0.455552,0.495311,0.449200
10,0.044100,2.272039,0.710075,0.449841,0.471159,0.444719


***** Running Evaluation *****
  Num examples = 25341
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-203
Configuration saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-203/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-203/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-203/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:3

Model weights saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-1827/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-1827/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-1827/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-1624] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 25341
  Batch size = 32
Saving model checkpoint to /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-2030
Configuration saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-2030/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL single task gender/20220923-20:32:02/checkpoint-2030/pytorch_model.bin
tokenizer co

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 25341
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# combine all prediction

In [ ]:
if save_prediction:
    df_combined = None
    for task in tasks2labels:
        df = pd.read_csv(f'{data_folder}/pastel/processed/p={p}_predicted/{task}.csv')
        df = df.rename({'logits': f'logits_{task}'}, axis=1)
        if df_combined is None:
            df_combined = df
        else:
            df_combined[task] = df[task]
            df_combined[f'logits_{task}'] = df[f'logits_{task}']
            df_combined[f'predicted_{task}'] = df[f'predicted_{task}']
    del df_combined['Unnamed: 0']
    df_combined.to_csv(f'{data_folder}/pastel/processed/p={p}_predicted/pastel.csv', index=False)    

In [ ]:
wandb.finish() 

eval/accuracy_age,▅█▇▄▃▃▁▁▃▁▂▄▁▂▃
eval/accuracy_country,██▁▅▆▆▇▇▇▇▇▇▇▇▇
eval/accuracy_education,▇▃█▅▃▁▂▂▂▂▂▃▄▃▃
eval/accuracy_ethnic,▇██▆▂▃▁▂▃▆▅▅▄▄▄
eval/accuracy_gender,▆▇█▅▁▆▄▄▆▅▆▅▆▅▅
eval/accuracy_politics,▆█▆▁▄▆▄▅▅▆▅▅▅▆▆
eval/accuracy_tod,▇█▅▁▁▃▃▃▄▃▃▃▃▃▃
eval/f1_age,▁▅▅▇▇▇▇▇▇██████
eval/f1_country,▁▁█▆▇▆▇▄▄▄▅▅▅▅▅
eval/f1_education,▁▅▆▆▇▆█▆▇███▇██
eval/f1_ethnic,▁▃▅▅▇▆█▇█▇▇▇▇▇▇
